# Setup Notebook Environment


In [2]:
val path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))


Compiling /Users/jeff/dev/coremem/core-mem-controller/Main.sc

Compiling /Users/jeff/dev/coremem/core-mem-controller/Main.sc #2

path: String = "/Users/jeff/dev/coremem/core-mem-controller/load-ivy.sc"

In [3]:

import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Top level

Instantiate the top level design module, and define the input ports

In [59]:
class CoreMem extends Module {
    val io = IO(new Bundle {
        // Control Bus
        val slv_addr = Input(UInt(3.W))
        val slv_rd = Input(Bool())
        val slv_wr = Input(Bool())
        val slv_data_o = Output(UInt(8.W))
        val slv_data_i = Input(UInt(8.W))
        val slv_data_t = Output(UInt(8.W))
        val slv_ack = Output(Bool())
        
        // Core driver signals
        val sense = Input(UInt(8.W))
        val sense_thresh = Output(Bool())
        val dir_f = Output(Bool())
        val dir_r = Output(Bool())
        val bit_drive_f = Output(UInt(8.W))
        val bit_drive_r = Output(UInt(8.W))
        val word_drive_f = Output(UInt(8.W))
        val word_drive_r = Output(UInt(8.W))
        val iset = Output(Bool())
    })
    
    io.slv_data_o := 0.U
    io.slv_data_t := 0xFF.U
    io.slv_ack := 0.U
    
    io.iset := 0.U
    io.sense_thresh := 0.U
    io.dir_f := 0.U
    io.dir_r := 0.U
    io.bit_drive_r := 0.U
    io.bit_drive_f := 0.U
    io.word_drive_r := 0.U
    io.word_drive_f := 0.U
    
//     def states = Map("idle" -> 0, "forward" -> 1, "forward-wait" -> 2,
//                      "reverse" -> 3, "reverse-wait" -> 4, "ack" -> 5)
    
    val idle :: forward :: forwardWait :: delay :: reverse :: reverseWait :: ack :: Nil = Enum(6)
    
    def PULSE_LENGTH = 100
    
    val next_state = UInt(3.W)
    val state = RegInit(idle)
    val drive_en = RegInit(false.B) 
    val drive_counter = RegInit(0.U(log2Ceil(PULSE_LENGTH).W))
    
    when(state === idle) {
        when(io.slv_wr || io.slv_rd) {
            next_state := forward
        }.otherwise {
            next_state := idle
        }
    }.elsewhen(state === forward) {
        next_state := forwardWait
    }.elsewhen(state === forwardWait) {
        when(!drive_en) {
            next_state := reverse
        }.otherwise {
            next_state := forwardWait
        }
    }.elsewhen(state === reverse) {
        next_state := reverseWait
    }.elsewhen(state === reverseWait) {
        when(!drive_en) {
            next_state := ack
        }.otherwise {
            next_state := reverseWait
        }
    }.elsewhen(state === ack) {
        // Hold ack until RD/WR are de-asserted by master
        when(!io.slv_rd && !io.slv_wr) {
            next_state := idle
        }.otherwise {
            next_state := ack
        }
    }.otherwise {
        next_state := idle
    }
    
    state := next_state
    
    val delayed_capture = Module(new DelayedCapture(10))
    
    when(next_state === forward || next_state === reverse) {
        drive_en := true.B
        drive_counter := PULSE_LENGTH.U
    }.otherwise {
        when(drive_counter > 0.U) {
            drive_counter := drive_counter - 1.U
        }.otherwise {
            drive_en := false.B
        }
    }
    
    
    val dir_is_forward = state === forward || state === forwardWait
    
    // Drive bit lines
    val write_value = UInt(8.W)
    val read_value = UInt(8.W)
    
    when(io.slv_wr) {
        // If writing, drive the data from the master bus
        write_value := io.slv_data_i
    }.otherwise {
        // When reading, drive the data we just read
        write_value := delayed_capture.io.out
    }
    
    
    io.bit_drive_f := 0.U
    io.bit_drive_r := 0.U
    when(drive_en) {
        when(dir_is_forward) {
            io.bit_drive_f := 0xff.U
        }.otherwise {
            io.bit_drive_r := write_value
        }
    }
    
    // Drive word line
    io.word_drive_f := 0.U
    io.word_drive_r := 0.U
    when(drive_en) {
        
        when(dir_is_forward) {
            io.word_drive_f := (1.U << io.slv_addr)
        }.otherwise {
            io.word_drive_r := (1.U << io.slv_addr)
        }
    }
    
    // Drive voltage thresholds
    val PWM_WIDTH = 11
    val PWM_CLKDIV = 1
    val iset_pwm = new PwmOutput(PWM_CLKDIV, PWM_WIDTH)
    val sense_thresh_pwm = new PwmOutput(PWM_CLKDIV, PWM_WIDTH)
    
    iset_pwm.io.duty := 0.U // TODO
    io.iset := iset_pwm.io.pwm
    sense_thresh_pwm.io.duty := 0.U // TODO
    io.sense_thresh := sense_thresh_pwm.io.pwm
    
}

defined class CoreMem

In [12]:
println(getVerilog(new CoreMem))


[info] [0.001] Elaborating design...
[info] [0.008] Done elaborating.
Total FIRRTL Compile Time: 183.2 ms
module cmd10HelperCoreMem(
  input        clock,
  input        reset,
  input  [2:0] io_slv_addr,
  input        io_slv_rd,
  input        io_slv_wr,
  output [7:0] io_slv_data_o,
  input  [7:0] io_slv_data_i,
  output [7:0] io_slv_data_t,
  output       io_slv_ack,
  input  [7:0] io_sense,
  output       io_sense_thresh,
  output       io_dir_f,
  output       io_dir_r,
  output [7:0] io_bit_drive_f,
  output [7:0] io_bit_drive_r,
  output [7:0] io_word_drive_f,
  output [7:0] io_word_drive_r,
  output       io_iset
);
  assign io_slv_data_o = 8'h0; // @[cmd10.sc 24:19]
  assign io_slv_data_t = 8'hff; // @[cmd10.sc 25:19]
  assign io_slv_ack = 1'h0; // @[cmd10.sc 26:16]
  assign io_sense_thresh = 1'h0; // @[cmd10.sc 29:21]
  assign io_dir_f = 1'h0; // @[cmd10.sc 30:14]
  assign io_dir_r = 1'h0; // @[cmd10.sc 31:14]
  assign io_bit_drive_f = 8'h0; // @[cmd10.sc 33:20]
  assign io_

In [45]:
class DelayedCapture(max_delay: Int, data_width: Int = 8) extends Module {
    val delay_width = log2Ceil(max_delay)
    val io = IO(new Bundle {
        val capture_delay = Input(UInt(delay_width.W))
        val trigger = Input(Bool())
        val in = Input(UInt(data_width.W))
        val out = Output(UInt(data_width.W))
        val valid = Output(Bool())
    })
    
    val running = RegInit(false.B)
    val outReg = RegInit(0.U(data_width.W))
    val counter = RegInit(0.U(delay_width.W))
    val validReg = RegInit(false.B)
    
    io.out := outReg
    io.valid := validReg
    
    def risingedge(x: Bool) = x && !RegNext(x)
    
    when(risingedge(io.trigger)) {
        running := true.B
    }
    
    validReg := false.B
    
    when(running) {
        
        val ovf = counter === (io.capture_delay - 1.U)
        counter := counter + 1.U
        when(ovf) {
            counter := 0.U
            running := false.B
            validReg := true.B
            outReg := io.in
        }
    }
}

defined class DelayedCapture

In [48]:
class DelayedCaptureTester(c: DelayedCapture) extends PeekPokeTester(c) {
    poke(c.io.capture_delay, 10)
    poke(c.io.in, 0)
    poke(c.io.trigger, 0)
    step(1)
    poke(c.io.trigger, 1)
    
    
    for(i <- 1 to 10) {
        expect(c.io.valid, false.B)
        step(1)
        poke(c.io.in, i)
    }
    
    step(1)
    
    expect(c.io.valid, true.B)
    expect(c.io.out, 10)
    
    step(1)
    expect(c.io.valid, false.B)
}

defined class DelayedCaptureTester

In [49]:
val works = iotesters.Driver(() => new DelayedCapture(10)) { c => new DelayedCaptureTester(c) }

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 17.9 ms
Total FIRRTL Compile Time: 16.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1556836659215
test cmd44HelperDelayedCapture Success: 13 tests passed in 18 cycles taking 0.013599 seconds
[info] [0.014] RAN 13 CYCLES PASSED


works: Boolean = true

In [51]:
class PwmOutput(clk_divisor: Int, counter_width: Int) extends Module {
    val io = IO(new Bundle {
        val duty = Input(UInt(counter_width.W))
        val pwm = Output(Bool())
    })
    
    val slow_en = RegInit(false.B)
    val div_counter = Counter(clk_divisor)
    val pwm_counter = RegInit(0.U(counter_width.W))
    
    slow_en := false.B
    
    when(div_counter.inc()) {
        slow_en := true.B
    }
    
    when(slow_en) {
        pwm_counter := pwm_counter + 1.U
    }
    
    io.pwm := RegNext(pwm_counter < io.duty)
}

defined class PwmOutput

In [58]:
class PwmOutputTester(c: PwmOutput, clk_divisor: Int, counter_width: Int) extends PeekPokeTester(c) {
    val maxCount = math.pow(2, counter_width).toInt
    val duty = maxCount / 4
    poke(c.io.duty, duty)
    
    for(i <- 0 to maxCount-1) {
        
        step(clk_divisor)
        if(i < duty) {
            expect(c.io.pwm, true)
        } else {
            expect(c.io.pwm, false)
        }
    }
}
val clk_divisor = 4
val counter_width = 12
val works = iotesters.Driver(() => new PwmOutput(clk_divisor, counter_width)) { 
    c => new PwmOutputTester(c, clk_divisor, counter_width) 
}

[info] [0.000] Elaborating design...
[info] [0.056] Done elaborating.
Total FIRRTL Compile Time: 13.7 ms
Total FIRRTL Compile Time: 12.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1556841567774
test cmd50HelperPwmOutput Success: 4096 tests passed in 16389 cycles taking 0.277462 seconds
[info] [0.277] RAN 16384 CYCLES PASSED


defined class PwmOutputTester
clk_divisor: Int = 4
counter_width: Int = 12
works: Boolean = true

In [43]:
class BusStateMachine extends Module {
    val io = IO(new Bundle {
        val wr = Input(Bool())
        val rd = Input(Bool())
        val ack = Output(Bool())
        
        val pulse_trigger = Output(Bool())
        val forward = Output(Bool())
        val pulse_complete = Input(Bool())
        
    })
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 24.1 ms
module cmd36HelperDelayedCapture(
  input        clock,
  input        reset,
  input        io_trigger,
  input  [7:0] io_in,
  output [7:0] io_out,
  output       io_valid
);
  reg  running; // @[cmd36.sc 9:26]
  reg [31:0] _RAND_0;
  reg [7:0] outReg; // @[cmd36.sc 10:25]
  reg [31:0] _RAND_1;
  reg [3:0] value; // @[Counter.scala 26:33]
  reg [31:0] _RAND_2;
  reg  validReg; // @[cmd36.sc 12:27]
  reg [31:0] _RAND_3;
  reg  _T_1; // @[cmd36.sc 17:44]
  reg [31:0] _RAND_4;
  wire  _T_2; // @[cmd36.sc 17:36]
  wire  _T_3; // @[cmd36.sc 17:33]
  wire  _T_4; // @[Counter.scala 34:24]
  wire [3:0] _T_6; // @[Counter.scala 35:22]
  assign _T_2 = _T_1 == 1'h0; // @[cmd36.sc 17:36]
  assign _T_3 = io_trigger & _T_2; // @[cmd36.sc 17:33]
  assign _T_4 = value == 4'h9; // @[Counter.scala 34:24]
  assign _T_6 = value + 4'h1; // @[Counter.scala 35:22]
  assign io_out = outReg; // @[cmd36.sc